In [ ]:
data_dir = '/group/jug/ashesh/naturemethods/weak_nucleus_exp/'

In [ ]:
! ls -l $data_dir

In [ ]:
import os
import pickle
from disentangle.core.tiff_reader import load_tiff
def get_mean_std():
    pkl_fpath = os.path.join(data_dir, 'data_stats.pkl')
    with open(pkl_fpath, 'rb') as f:
        mean_std = pickle.load(f)
    return mean_std

def unnormalize(data, mean_std):
    mean = mean_std['mean']['target'].squeeze()[None,None,None]
    std = mean_std['std']['target'].squeeze()[None,None,None]
    return data * std + mean

def get_data():
    mean_std = get_mean_std()
    no_shift = {
        'gt': load_tiff(os.path.join(data_dir, 'gt_no_shift.tif')),
        'pred': load_tiff(os.path.join(data_dir, 'pred_no_shift.tif')),
    }
    shift = {
        'gt': load_tiff(os.path.join(data_dir, 'gt_shifted.tif')),
        'pred': load_tiff(os.path.join(data_dir, 'pred_shifted.tif')),
    }
    zeroed = {
        'gt': load_tiff(os.path.join(data_dir, 'gt_zeroed.tif')),
        'pred': load_tiff(os.path.join(data_dir, 'pred_zeroed.tif')),
    }
    no_shift['pred'] = unnormalize(no_shift['pred'], mean_std)
    shift['pred'] = unnormalize(shift['pred'], mean_std)
    zeroed['pred'] = unnormalize(zeroed['pred'], mean_std)
    return no_shift, shift, zeroed


In [ ]:
no_shift, shift, zeroed = get_data()

In [ ]:
no_shift['gt'][0].mean(axis=-1).shape

In [ ]:
no_shift['gt'].min()

In [ ]:
# import matplotlib.pyplot as plt
# from disentangle.analysis.plot_utils import clean_ax
# from matplotlib.colors import LogNorm

# ncols = 5
# nrows = 3
# hs = 1500
# he = 2500
# ws = 1500
# we = 4096-700
# h = he - hs
# w = we - ws
# factor = h/w
# norm = LogNorm(vmin=0.01, vmax=1541)
# _,ax = plt.subplots(ncols=ncols,nrows=nrows,figsize=(ncols*4,int(nrows*4*factor)))
# ax[0,0].imshow(no_shift['gt'][0].mean(axis=-1)[hs:he,ws:we], cmap='magma',norm=norm)
# ax[0,1].imshow(no_shift['gt'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[0,2].imshow(no_shift['pred'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[0,3].imshow(no_shift['gt'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)
# ax[0,4].imshow(no_shift['pred'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)

# ax[1,0].imshow(shift['gt'][0].mean(axis=-1)[hs:he,ws:we], cmap='magma', norm=norm)
# ax[1,1].imshow(shift['gt'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[1,2].imshow(shift['pred'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[1,3].imshow(shift['gt'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)
# ax[1,4].imshow(shift['pred'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)

# ax[2,0].imshow(zeroed['gt'][0].mean(axis=-1)[hs:he,ws:we], cmap='magma', norm=norm)
# ax[2,1].imshow(zeroed['gt'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[2,2].imshow(zeroed['pred'][0,...,0][hs:he,ws:we], cmap='magma', norm=norm)
# ax[2,3].imshow(zeroed['gt'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)
# ax[2,4].imshow(zeroed['pred'][0,...,1][hs:he,ws:we], cmap='magma', norm=norm)
# clean_ax(ax)
# plt.subplots_adjust(wspace=0.01, hspace=0.01)


In [ ]:
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax
from matplotlib.colors import LogNorm
from disentangle.core.psnr import RangeInvariantPsnr
dict_type = 'shift'
assert dict_type in ['no_shift', 'shift', 'zeroed']



data_dict = eval(dict_type)
ncols = 5
nrows = 2

hs = 1500
he = 2500
hm = (hs+he)//2
ws = 1500
we = 4096-700
def loc_to_str():
    return f'{hs}-{he}_{ws}-{we}'

h = he - hs
w = we - ws
factor = h/w
norm = LogNorm(vmin=0.01, vmax=1541)
_,ax = plt.subplots(ncols=ncols,nrows=nrows,figsize=(ncols*4 -1,int(nrows*4*factor/2)))
ax[0,0].imshow(data_dict['gt'][0].mean(axis=-1)[hs:hm,ws:we], cmap='magma',norm=norm)
ax[0,1].imshow(data_dict['gt'][0,...,0][hs:hm,ws:we], cmap='magma', norm=norm)
ax[0,2].imshow(data_dict['pred'][0,...,0][hs:hm,ws:we], cmap='magma', norm=norm)
ax[0,3].imshow(data_dict['gt'][0,...,1][hs:hm,ws:we], cmap='magma', norm=norm)
ax[0,4].imshow(data_dict['pred'][0,...,1][hs:hm,ws:we], cmap='magma', norm=norm)

ax[1,0].imshow(data_dict['gt'][0].mean(axis=-1)[hm:he,ws:we], cmap='magma',)

vmin = data_dict['gt'][0,...,0][hm:he,ws:we].min()
vmax = data_dict['gt'][0,...,0][hm:he,ws:we].max()
ax[1,1].imshow(data_dict['gt'][0,...,0][hm:he,ws:we], cmap='magma',vmin=vmin, vmax=vmax)
ax[1,2].imshow(data_dict['pred'][0,...,0][hm:he,ws:we], cmap='magma',vmin=vmin, vmax=vmax)

vmin = data_dict['gt'][0,...,1][hm:he,ws:we].min()
vmax = data_dict['gt'][0,...,1][hm:he,ws:we].max()
ax[1,3].imshow(data_dict['gt'][0,...,1][hm:he,ws:we], cmap='magma',vmin=vmin, vmax=vmax)
ax[1,4].imshow(data_dict['pred'][0,...,1][hm:he,ws:we], cmap='magma',vmin=vmin, vmax=vmax )

clean_ax(ax)
plt.subplots_adjust(wspace=0.0, hspace=0.0)
fname = f'{dict_type}_{loc_to_str()}.png'
fpath = os.path.join(data_dir,fname)
print(RangeInvariantPsnr(data_dict['gt'][...,0], data_dict['pred'][...,0]),
RangeInvariantPsnr(data_dict['gt'][...,1], data_dict['pred'][...,1]))
print(fpath)
# plt.tight_layout()
plt.savefig(fpath, dpi=200)


In [ ]:
import pandas as pd
_,ax = plt.subplots(1,1,figsize=(4/1.6,4*factor))
pd.DataFrame(data_dict['gt'][0][hm+100, ws:we], columns=['Actin','Nucleus']).plot(ax=ax, fontsize=12).legend(fontsize=10)
fname = f'{dict_type}_lineplot_{loc_to_str()}.png'
fpath = os.path.join(data_dir,fname)
print(fpath)
plt.tight_layout()
plt.savefig(fpath, dpi=200)
